In [1]:
import os
import pandas as pd
import gtfs #homemade module
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('spark_parse/test2.txt',header=None)

/opt/rh/anaconda/root/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
# beware!  Bonan is still working on organizing the extract files.  these columns may change.
bustime.columns = ['route','lat','lon','timestamp','vehicle_id','trip_id','trip_date','shape_id','next_stop_id','dist_from_stop','stop_dist_on_trip','presentable_dist']

In [4]:
bustime.drop_duplicates(['vehicle_id','timestamp'],inplace=True)
bustime['trip_id'] = bustime['trip_id'].str.replace('MTA NYCT_','')
bustime['trip_id'] = bustime['trip_id'].str.replace('MTABC_','')
bustime.set_index(['route','trip_id','trip_date','vehicle_id','timestamp'],
                  inplace=True,drop=True,verify_integrity=True)

In [5]:
# for demonstration, use a subset. Just get data for one line (M5) on one day.
tripDateLookup = "2015-01-06"
lineLookup = "MTA NYCT_B41"
bustime = bustime.xs((lineLookup,tripDateLookup),level=(0,2),
                           drop_level=False)
# note that the AVL dataframe must be sorted by timestammp, since iloc[]
# selection is used later in this script to find the earliest time
bustime.sort_index(inplace=True)
print 'Finished loading BusTime data and and slicing B41 line.'

Finished loading BusTime data and and slicing B41 line.


In [6]:
# be careful - looks like pandas did not recognize this column to be numeric data
bustime.iloc[1].stop_dist_on_trip

'656.78'

In [7]:
bustime['stop_dist_on_trip'] = bustime['stop_dist_on_trip'].convert_objects(convert_numeric=True)
bustime['dist_from_stop'] = bustime['dist_from_stop'].convert_objects(convert_numeric=True)

In [8]:
bustime['veh_dist_along_trip'] = bustime['stop_dist_on_trip'] - bustime['dist_from_stop']

In [9]:
bustime['veh_dist_along_trip']

route         trip_id                            trip_date   vehicle_id     timestamp                    
MTA NYCT_B41  FB_A5-Weekday-SDon-004500_B41_1    2015-01-06  MTA NYCT_4233  2015-01-06T00:39:55.000-05:00      618.74
                                                                            2015-01-06T00:40:58.000-05:00      618.74
                                                                            2015-01-06T00:42:02.000-05:00      618.74
                                                                            2015-01-06T00:45:41.000-05:00      239.10
                                                                            2015-01-06T00:46:43.000-05:00      327.11
                                                                            2015-01-06T00:47:45.000-05:00      555.35
                                                                            2015-01-06T00:48:49.000-05:00      764.02
                                                                    

In [12]:
stop_distances = bustime.reset_index().drop_duplicates(['shape_id','next_stop_id','stop_dist_on_trip'])
stop_distances.shape_id = stop_distances.shape_id.str.replace('MTA_','')
stop_distances.next_stop_id = stop_distances.next_stop_id.str.replace('MTA_','')
stop_distances.drop(['route','trip_id','trip_date','vehicle_id','timestamp','lat','lon','dist_from_stop','presentable_dist','veh_dist_along_trip'],axis=1,inplace=True)

In [13]:
# verify that shape_id and next_stop_id is a valid primary key (that is, exactly one value for each index)
sum(stop_distances.duplicated(['shape_id','next_stop_id']))

0

In [14]:
stop_distances = stop_distances.sort(['shape_id','stop_dist_on_trip']).reset_index(drop=True)
stop_distances.head()

,shape_id,next_stop_id,stop_dist_on_trip
0,B410227,303270,334.47
1,B410227,303274,950.80
2,B410227,303276,1283.55
3,B410227,303277,1472.70
4,B410227,303222,2087.51


In [15]:
# mean distance between stops on trips
stop_distances.groupby('shape_id')['stop_dist_on_trip'].diff().mean()

375.39760814249365